# 🥋 Lekcja 25: Weight Sharing (Wiązanie Wag)

Wiązanie wag to technika, gdzie ten sam tensor parametrów jest używany w kilku miejscach sieci.

**Zastosowania:**
1.  **Autoenkodery:** Wagi Dekodera są transpozycją Encodera.
2.  **NLP (Language Models):** Warstwa Embeddingu (wejście) i warstwa projekcji (wyjście) często współdzielą tę samą macierz (wielką, np. 50k słów).
3.  **Sieci Syjamskie:** Dwa obrazki przechodzą przez *tę samą* sieć (to też forma weight sharingu).

**Jak to zrobić w PyTorch?**
Po prostu przypisz ten sam obiekt `nn.Parameter` do dwóch atrybutów.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Podejście Standardowe (Niezależne wagi)
class StandardAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim) # Nowa, niezależna macierz

    def forward(self, x):
        x = F.relu(self.encoder(x))
        x = self.decoder(x)
        return x

model_std = StandardAutoencoder(10, 5)

print("--- STANDARD ---")
print(f"Adres wagi Encoder: {model_std.encoder.weight.data_ptr()}")
print(f"Adres wagi Decoder: {model_std.decoder.weight.data_ptr()}")
print("Adresy są RÓŻNE. To dwie osobne macierze.")

--- STANDARD ---
Adres wagi Encoder: 3704677794240
Adres wagi Decoder: 3704677794560
Adresy są RÓŻNE. To dwie osobne macierze.


## Implementacja Tied Weights

Teraz zrobimy to sprytnie.
Nie będziemy tworzyć `nn.Linear` dla decodera.
Użyjemy `nn.Linear` dla Encodera, a w Decoderze użyjemy **funkcyjnego** wywołania `F.linear`, podając mu wagę Encodera (transponowaną).

In [2]:
class TiedAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        # Encoder: Standardowa warstwa
        self.encoder = nn.Linear(input_dim, hidden_dim)
        
        # Decoder: NIE tworzymy nowej warstwy Linear.
        # Możemy ewentualnie stworzyć osobny bias, bo bias nie musi być wiązany
        self.decoder_bias = nn.Parameter(torch.zeros(input_dim))

    def forward(self, x):
        # 1. Encode
        x = F.relu(self.encoder(x))
        
        # 2. Decode (Używamy wagi Encodera!)
        # Wzór: x @ W_enc.T + bias
        # F.linear robi x @ W.T, więc my musimy podać W_enc.T.T = W_enc? 
        # Czekaj! F.linear(input, weight) wykonuje: input @ weight.T
        # My chcemy użyć tej samej macierzy, ale transponowanej względem Encodera.
        # Waga Encodera ma kształt [Hidden, Input].
        # Do Decodera potrzebujemy [Input, Hidden] (żeby pomnożyć [Batch, Hidden]).
        # F.linear oczekuje wagi o kształcie [Out, In].
        
        # Tutaj waga encoder.weight ma kształt [Hidden, Input].
        # My chcemy wyjście o rozmiarze Input.
        # F.linear(x, self.encoder.weight.t()) -> x @ (W.t()).t() -> x @ W
        
        # Dla Tied Weights w AE zazwyczaj używamy: W_dec = W_enc.T
        x = F.linear(x, self.encoder.weight.t(), self.decoder_bias)
        
        return x

model_tied = TiedAutoencoder(10, 5)

print("\n--- TIED WEIGHTS ---")
print(f"Waga Encodera: {model_tied.encoder.weight.shape}")
# Nie ma 'model_tied.decoder', bo użyliśmy F.linear
print("Wagi są fizycznie tym samym obiektem.")


--- TIED WEIGHTS ---
Waga Encodera: torch.Size([5, 10])
Wagi są fizycznie tym samym obiektem.


## Dowód: Gradienty

Skoro wagi są współdzielone, to podczas `backward()` gradienty z "obu stron" (z wejścia i wyjścia) powinny się **zsumować** w jednym parametrze.

Zróbmy test:
1.  Puścimy dane.
2.  Policzymy stratę.
3.  Zrobimy `backward`.
4.  Sprawdzimy, czy waga Encodera otrzymała gradient.

In [3]:
x = torch.randn(1, 10)
y = x.clone() # Autoenkoder ma odtworzyć wejście

# Forward
out = model_tied(x)
loss = ((out - y)**2).sum()

# Backward
loss.backward()

print("--- GRADIENTY ---")
print(f"Czy waga Encodera ma gradient? {model_tied.encoder.weight.grad is not None}")
print(f"Wartość gradientu (norma): {model_tied.encoder.weight.grad.norm().item():.4f}")

# Zmieńmy wagę ręcznie i sprawdźmy czy 'decoder' (logika) to odczuje
with torch.no_grad():
    model_tied.encoder.weight.fill_(10.0)

out_new = model_tied(x)
print(f"\nPo zmianie wagi Encodera, wynik Decodera: {out_new.mean().item():.2f}")
print("(Wynik jest ogromny, co dowodzi, że zmiana wagi wpłynęła na całą sieć).")

--- GRADIENTY ---
Czy waga Encodera ma gradient? True
Wartość gradientu (norma): 7.5713

Po zmianie wagi Encodera, wynik Decodera: 1769.00
(Wynik jest ogromny, co dowodzi, że zmiana wagi wpłynęła na całą sieć).


## Alternatywa: Przypisanie atrybutu

Można też zrobić to bardziej "obiektowo", przypisując ten sam parametr do dwóch warstw `nn.Linear`.

In [4]:
class SharedLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 10)
        
        # NADPISUJEMY wagę fc2 wagą fc1
        self.fc2.weight = self.fc1.weight
        # Teraz to ten sam obiekt w pamięci!

model_shared = SharedLinear()

print(f"FC1 ptr: {model_shared.fc1.weight.data_ptr()}")
print(f"FC2 ptr: {model_shared.fc2.weight.data_ptr()}")

if model_shared.fc1.weight.data_ptr() == model_shared.fc2.weight.data_ptr():
    print("✅ Adresy identyczne. PyTorch będzie aktualizował oba naraz.")

FC1 ptr: 3704678056576
FC2 ptr: 3704678056576
✅ Adresy identyczne. PyTorch będzie aktualizował oba naraz.


## 🥋 Black Belt Summary

1.  **Weight Sharing** to nie magia. To po prostu użycie tego samego tensora w wielu miejscach grafu obliczeniowego.
2.  **Autograd** radzi sobie z tym doskonale. Gradienty z różnych miejsc grafu, które używają tej samej wagi, są po prostu **sumowane** (Accumulate Grad).
3.  **Zastosowanie:**
    *   Zmniejszenie liczby parametrów (mniejszy model).
    *   Regularyzacja (trudniej o overfitting, gdy wagi są związane).
    *   W NLP (Transformers) wiązanie Embeddingu wejściowego z wyjściowym to standard (Input/Output Embedding Tying).